In [1]:
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss
import numpy as np

%matplotlib inline

In [2]:
class GCF:
    N_FOLDS = 5
    SEED = 0

In [3]:
train_df = pd.read_csv("data/train.csv")
test_df = pd.read_csv("data/test.csv")
sub = pd.read_csv("data/sample_submission.csv")

In [5]:
skf = StratifiedKFold(n_splits=GCF.N_FOLDS, random_state=GCF.SEED, shuffle=True).split(train_df.index, train_df['fav_novel_cnt_bin'].values)
for fold, (_, valid_index) in enumerate(skf):
    train_df.loc[valid_index, 'fold'] = fold

In [6]:
bin_feat_name = ['end', 'isstop', 'isr15', 'isbl', 'isgl', 'iszankoku', 'istensei', 'istenni']

def get_category_features(fit_df):
    new_feat_df = pd.DataFrame({
        'novel_type_bin': fit_df['novel_type'].map(lambda x: x - 1),
        'write_ketai': fit_df['pc_or_k'].map(lambda x: int(x in [1, 3])),
        'write_pc': fit_df['pc_or_k'].map(lambda x: int(x in [2, 3])),
    })

    biggenre_df = pd.get_dummies(fit_df['biggenre'])
    biggenre_df.columns = [f"biggenre_{c}" for c in biggenre_df.columns]

    genre_df = pd.get_dummies(fit_df['genre'])
    genre_df.columns = [f"genre_{c}" for c in genre_df.columns]

    cate_feat_df = pd.concat([
        fit_df[bin_feat_name],
        new_feat_df,
        biggenre_df,
        genre_df
    ], axis=1)
    
    return cate_feat_df

In [7]:
import MeCab
tagger = MeCab.Tagger("-Ochasen")

In [8]:
text ='暗い窓辺と皇帝の北の宮'

def sp_mecab(text):
    wakati = [t for t in tagger.parse(text).split('\n')  if t not in ["", "EOS"]]
    sp_text = [w.split('\t')[0] for w in wakati]
    return ' '.join(sp_text)

In [9]:
from sklearn.feature_extraction.text import CountVectorizer

In [10]:
corpus = train_df['title'].map(sp_mecab).tolist()

In [11]:
vectorizer = CountVectorizer()
bow = vectorizer.fit_transform(corpus)

title_bow_df =pd.DataFrame(bow.toarray(), columns=vectorizer.get_feature_names())

/usr/local/var/pyenv/versions/3.8.10/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [12]:
X = pd.concat([
    get_category_features(train_df),
    title_bow_df,
], axis=1)
y = train_df['fav_novel_cnt_bin']

In [15]:
oof = np.zeros((len(train_df), 5))
for fold in range(GCF.N_FOLDS):
    train_idx = train_df.query(f'fold!={fold}').index
    valid_idx = train_df.query(f'fold=={fold}').index
    
    X_train = X.loc[train_idx].values
    X_valid = X.loc[valid_idx].values
    y_train = y.loc[train_idx].values
    y_valid = y.loc[valid_idx].values
    
    clf = LogisticRegression()
    clf.fit(X_train, y_train)
    
    pred_train = clf.predict_proba(X_train)
    pred_valid = clf.predict_proba(X_valid)
    
    oof[valid_idx, :] = pred_valid
    break

/usr/local/var/pyenv/versions/3.8.10/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
(y_valid.values == pred_valid.argmax(1)).sum() / len(y_valid)

In [ ]:
pred_train

In [ ]:
(pred_train * np.array([np.eye(5)[i] for i in y_train])).sum(1)

In [ ]:
val_pred = model.predict(val_x, num_iteration=model.best_iteration)
val_pred_max = np.argmax(val_pred, axis=1)  # 最尤と判断したクラスの値にする
accuracy = sum(val_y == val_pred_max) / len(val_y)
print(accuracy)

In [ ]:
reg = LinearRegression()
reg.fit(X_train, y_train)

In [ ]:
pd.DataFrame(reg.predict(X_valid)).hist(bins=100)

In [93]:
from sklearn.linear_model import LinearRegression